# Projeto 1

### João Antônio Misson Milhorim - 11834331
### Reynaldo Coronatto - 

In [1]:
#%pip install glfw
#%pip install PyOpenGL
#%pip install numpy
#%pip install glm
#%pip install Pillow

In [2]:
import os
import math
import glfw
import random
import numpy as np
import OpenGL.GL.shaders

from PIL import Image
from OpenGL.GL import *

In [3]:
class GLWrapper:
    ''' Classe que faz o setup do OpenGL e permite a visualização de imagens '''
    tx, ty, sx, sy = 0.0, 0.0, 1.0, 1.0
    theta = np.radians(0)
        
    def __init__(self, height=700, width=700, title='', figures=None):
        ''' Construtor responsável por chamar funções que configuram o OpenGL '''
        #self.show_light = False
        #self.figures = figures
        #self.set_keys()
        self.window = self.init_window(height, width, title)
        #self.add_events(self.window)
        self.program, self.vertex, self.fragment = self.set_shaders()
        self.compile_shaders(self.vertex, 'vertex')
        self.compile_shaders(self.fragment, 'fragment')
        self.attach_shaders(self.program, self.vertex, self.fragment)
        self.link_program(self.program)
    
    def init_window(self, height, width, title):
        ''' Inicializa janela '''
        glfw.init()
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
        window = glfw.create_window(height, width, title, None, None)
        glfw.make_context_current(window)
        return window
    
    def set_shaders(self):
        ''' Adiciona shaders '''
        vertex_code = """
                attribute vec3 position;
                attribute vec2 texture_coord;
                varying vec2 out_texture;

                uniform mat4 mat_transform;        

                void main(){
                    gl_Position = mat_transform * vec4(position,1.0);
                    out_texture = vec2(texture_coord);
                }
                """

        fragment_code = """
                uniform vec4 color;
                varying vec2 out_texture;
                uniform sampler2D samplerTexture;

                void main(){
                    vec4 texture = texture2D(samplerTexture, out_texture);
                    gl_FragColor = texture;
                }
                """

        program  = glCreateProgram()
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        return program, vertex, fragment

    def compile_shaders(self, shader, name):
        ''' Compila shaders '''
        glCompileShader(shader)
        if not glGetShaderiv(shader, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(shader).decode()
            print(error)
            raise RuntimeError(f"Erro de compilacao do {name} Shader")
            
    def attach_shaders(self, program, vertex, fragment):
        ''' Associa shaders '''
        glAttachShader(program, vertex)
        glAttachShader(program, fragment)
        
    def link_program(self, program):
        ''' Faz o link do programa '''
        glLinkProgram(program)
        if not glGetProgramiv(program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(program))
            raise RuntimeError('Linking error')

        glUseProgram(program)
    
    def insert_figure_print(self, vertices, mat_transform=None):
        ''' Gera um buffer e insere dados vertices na janela, utiliza matriz
            de transformação, caso seja informada '''
        buffer = glGenBuffers(1)
        glBindBuffer(GL_ARRAY_BUFFER, buffer)

        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
        glBindBuffer(GL_ARRAY_BUFFER, buffer)

        stride = vertices.strides[0]
        offset = ctypes.c_void_p(0)

        loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(loc)

        glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

        if mat_transform is not None:
            loc = glGetUniformLocation(self.program, "mat_transformation")
            glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
        
    """
    def get_mat_scale(self, fig_name, xf, yf, pos):
        ''' Gera matriz que realiza alteração da escala da imagem 
            de acordo aos parametros da figura especificada '''
        if pos is not None:
            s = self.figures[fig_name][pos]['s']
        else:
            s = self.figures[fig_name]['s']
            
        mat_scale = np.array([[s, 0.0, 0.0, xf*(1-s)], 
                              [0.0, s, 0.0, yf*(1-s)], 
                              [0.0, 0.0, 0.0, 0.0], 
                              [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_scale

    def get_mat_rotate(self, fig_name, xr, yr, pos):
        ''' Gera matriz que realiza alteração de rotação na imagem 
            de acordo aos parametros da figura especificada'''
        if pos is not None:
            theta = self.figures[fig_name][pos]['theta']
        else:
            theta = self.figures[fig_name]['theta']
            
        mat_rotation = np.array([[np.cos(theta), -np.sin(theta), 0.0, xr-xr*np.cos(theta)+yr*np.sin(theta)], 
                                 [np.sin(theta), np.cos(theta), 0.0, yr-yr*np.cos(theta)-xr*np.sin(theta)], 
                                 [0.0, 0.0, 1.0, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_rotation
    
    def get_mat_translate(self, fig_name, pos):
        ''' Gera matriz que realiza alteração de translação na imagem 
            de acordo aos parametros da figura especificada'''
        if pos is not None:
            tx = self.figures[fig_name][pos]['tx']
            ty = self.figures[fig_name][pos]['ty']
        else:
            tx = self.figures[fig_name]['tx']
            ty = self.figures[fig_name]['ty']
            
        mat_translation = np.array([[1.0, 0.0, 0.0, tx], 
                                    [0.0, 1.0, 0.0, ty], 
                                    [0.0, 0.0, 1.0, 0.0], 
                                    [0.0, 0.0, 0.0, 1.0]], np.float32)
        return mat_translation
    
    def get_mat_transform(self, fig_name, xr=0, yr=0, xf=0, yf=0, pos=None):
        ''' Gera matriz que realiza a transformação final na imagem 
            combinando as demais matrizes de acordo aos parametros da figura especificada'''
        mat_transform =  self.get_mat_rotate(fig_name, xr, yr, pos) @ \
                         self.get_mat_translate(fig_name, pos) @ \
                         self.get_mat_scale(fig_name, xf, yf, pos)
        return mat_transform
    
    def set_keys(self):
        self.KEY_SPACE = 32
        self.KEY_ARROW_DOWN = 264
        self.KEY_ARROW_UP = 265
        self.KEY_ARROW_LEFT = 32
        self.KEY_ARROW_RIGHT = 262
        self.KEY_A = 65
        self.KEY_S = 83
        self.KEY_D = 68
        self.KEY_W = 87
        self.MOUSE_LEFT = 0
        self.MOUSE_RIGHT = 1
        
    def key_event(self, window, key, scancode, action, mods):
        ''' Callback para cliques no teclado '''
        if action == 0:
            return

        delta = 0.02
        button = 'Desconhecido'
        
        if key == self.KEY_W: # sobe ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['ty'] += delta
            button = 'W'
        
        if key == self.KEY_S: # desce ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['ty'] -= delta
            button = 'S'
        
        if key == self.KEY_A: # ovni para a esquerda
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['tx'] -= delta
            button = 'A'
        
        if key == self.KEY_D: # ovni para direita
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['tx'] += delta
            button = 'D'
        
        if key == self.KEY_ARROW_RIGHT: # rotaciona sol e lua
            for i in range(len(self.figures['sun'])):
                self.figures['sun'][i]['theta'] -= delta
            for i in range(len(self.figures['moon'])):
                self.figures['moon'][i]['theta'] -= delta
            button = 'Seta direita'
        
        if key == self.KEY_ARROW_UP: # aumenta ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['s'] += delta
            button = 'Seta cima'
        
        if key == self.KEY_ARROW_DOWN: # diminui ovni
            for i in range(len(self.figures['ufo'])):
                self.figures['ufo'][i]['s'] -= delta
            button = 'Seta baixo'
        
        if key == self.KEY_SPACE: # esconde ou mostra a luz do ovni
            self.show_light = not self.show_light

    def mouse_event(self, window, button, action, mods):
        ''' Callback para cliques no mouse '''
        if action == 0:
            return 
        
        button_str = 'Desconhecido'
        if button == self.MOUSE_LEFT: # carro para a esquerda
            for i in range(len(self.figures['car'])):
                self.figures['car'][i]['tx'] -= 0.02
            button_str = 'Botão Esquerdo'
        
        if button == self.MOUSE_RIGHT: # carro para a direita
            for i in range(len(self.figures['car'])):
                self.figures['car'][i]['tx'] += 0.02
            button_str = 'Botão Direito'
    
    def add_events(self, window):
        ''' Adiciona callbacks dos eventos '''
        glfw.set_key_callback(window, self.key_event)
        glfw.set_mouse_button_callback(window, self.mouse_event)
    """

In [ ]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []
    max_values = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))
    
    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)
    

def read_objects_from_dir():
    # Define the path to the 'obj' folder
    obj_folder_path = os.path.join(os.path.dirname('./'), 'obj')

    # Initialize the 'models' dictionary
    models = {}

    # Iterate through each folder in 'obj'
    for folder_name in os.listdir(obj_folder_path):
        folder_path = os.path.join(obj_folder_path, folder_name)

        # Check if the current item in 'obj' is a directory
        if os.path.isdir(folder_path):
            # Initialize a dictionary for the current folder
            folder_data = {}

            # Look for '.obj' and '.png' files in the current folder
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)

                folder_data['name'] = folder_name
                folder_data['visible'] = False
                if file_name.endswith('.obj'):
                    folder_data['obj'] = file_path
                elif file_name.endswith('.jpg'):
                    folder_data['tex'] = file_path

            # Add the folder's data to the 'models' dictionary
            models[len(models)] = folder_data

    # Print the resulting 'models' dictionary
    return models    



def load_model(model_id):
    modelo = load_model_from_file(models[model_id]['obj'])

    ### inserindo vertices do modelo no vetor de vertices
    print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
    models[model_id]['init_vertex'] = len(vertices_list)
    for face in modelo['faces']:
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
    print('Processando modelo cube.obj. Vertice final:',len(vertices_list))
    models[model_id]['end_vertex'] = len(vertices_list)

    ### inserindo coordenadas de textura do modelo no vetor de texturas


    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    load_texture_from_file(model_id, models[model_id]['tex'])
    
    
def desenha_obj(model_id):
           
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_id)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS , models[model_id]['init_vertex'], models[model_id]['end_vertex']) ## renderizando
    
    
    
def key_event(window,key,scancode,action,mods):
    global s_inc, tx_inc, ty_inc, rx_inc, ry_inc, polygonal_mode, models
    
    if key == 87:# and action == 1: # tecla W
        ty_inc += 0.001
    
    if key == 83: # tecla S
        ty_inc -= 0.001
    
    if key == 65: # tecla A
        tx_inc -= 0.001
        
    if key == 68: # tecla D
        tx_inc += 0.001
    
    if key == 90: # tecla Z
        s_inc -= 0.0001
    
    if key == 88: # tecla X
        s_inc += 0.0001
    
    if key == 262: # seta direita
        rx_inc += 0.01
    
    if key == 263: # seta esquerda
        rx_inc -= 0.01
    
    if key == 264: # seta baixo
        ry_inc -= 0.01
    
    if key == 265: # seta cima
        ry_inc += 0.01
        
    if key == 80 and polygonal_mode == True:
        polygonal_mode = False
    
    elif key == 80 and polygonal_mode == False:
        polygonal_mode = True
        
    for model_id in models:  
        if key == 49 + int(model_id) and models[model_id]['visible'] == False: # tecla 1
            models[model_id]['visible'] = True

        elif key == 49 + int(model_id) and models[model_id]['visible'] == True: # tecla 1
            models[model_id]['visible'] = False

In [4]:
g = GLWrapper(title='Projeto 1')


glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)


vertices_list = []    
textures_coord_list = []


read_objects_from_dir


for model_id in models:
    print(model_id)
    load_model(model_id)
    print(models[model_id])
    
    
    
    
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(g.program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(g.program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


polygonal_mode = False
s_inc = 0.0
tx_inc = 0.0
ty_inc = 0.0

rx_inc = 0.0
ry_inc = 0.0


glfw.set_key_callback(g.window, key_event)

In [19]:
glfw.show_window(g.window)

glEnable(GL_DEPTH_TEST) ### importante para 3D

from numpy import random


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

rotacao_inc = 0
d = 0 

t_x = 0.0
t_y = 0.0

scale = 0.01


while not glfw.window_should_close(g.window):

    glfw.poll_events() 
    
    d -= 0.001 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)

    t_x += tx_inc
    t_y += ty_inc

    scale += s_inc
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode == True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    if polygonal_mode == False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    mat_rotation_z = np.array([     1.0, -0.0, 0.0, 0.0, 
                                    0.0,  1.0, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, 1.0, -0.0, 0.0, 
                                    0.0, 0.0,  1.0, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     1.0,  0.0, 0.0, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_scale       = np.array([    scale, 0.0, 0.0, 0.0, 
                                    0.0, scale, 0.0, 0.0, 
                                    0.0, 0.0, scale, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    

    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_y)
    mat_transform = multiplica_matriz(mat_rotation_x,mat_transform)
    mat_transform = multiplica_matriz(mat_scale,mat_transform)
    mat_transform = multiplica_matriz(mat_translation, mat_transform)


    loc_mat_transform = glGetUniformLocation(g.program, "mat_transform")
    glUniformMatrix4fv(loc_mat_transform, 1, GL_TRUE, mat_transform) 

    for model_id in models:
        if models[model_id]['visible'] == True:
            desenha_obj(model_id)

    
    glfw.swap_buffers(g.window)

glfw.terminate()